# IMPORT

In [1]:
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.common.by import By
import googlemaps
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
import numpy as np

# SCRAPE LINKS

In [3]:
# create a new Chrome browser instance
service = Service(executable_path='D:\Downloads\chromedriver_win32')
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service = service, options=options)

In [4]:
# navigate to the application home page
url = 'https://www.google.com/maps'
driver.get(url)
time.sleep(5)

In [20]:
def get_links(district):
    # find the search box element on the page
    search_box = driver.find_element(By.ID, 'searchboxinput')

    # type district into the search box
    search_box.clear()
    search_box.send_keys(district)

    # find the search button
    search_button = driver.find_element(By.ID, 'searchbox-searchbutton')
    # click search button
    search_button.click()
    time.sleep(5)
    # type "nhà hàng" into the search box
    search_box.clear()
    search_box.send_keys('nhà hàng')

    # find the search button
    search_button = driver.find_element(By.ID, 'searchbox-searchbutton')
    # click search button
    search_button.click()
    time.sleep(5)
    SCROLL_PAUSE_TIME = 5
    # Get div result
    div = driver.find_element(By.CLASS_NAME, 'hfpxzc')
    # scroll down 
    for i in range(0, 15):  
        for j in range(0, 10):
            div.send_keys(Keys.PAGE_DOWN) #press PAGE_DOWN
        time.sleep(SCROLL_PAUSE_TIME) #ưait for loading
    # get all links
    links = [e.get_attribute('href') for e in driver.find_elements(By.CLASS_NAME, 'hfpxzc')]
    districts = [district for i in range(len(links))]
    links = pd.DataFrame({'district': districts, 'link': links})
    return links

In [23]:
df = pd.DataFrame()
distric_list = ['Phú Nhuận', 'Bình Thạnh', 'Tân Bình', 'Quận 5', 'Quận 4']
for district in distric_list:
    df = pd.concat([df, get_links(district)], axis = 0)
    df.to_csv('data/restaurant.csv', index = False)
    print(len(df))
driver.quit()

122
244
366
488
610



# Scrape Reviews

In [11]:
df = pd.read_csv('restaurant_link.csv')
df = df.dropna()
df = df.drop_duplicates(subset = ['link'])
df.info()
df.to_csv('restaurant_link.csv', index = False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 305 entries, 0 to 608
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   district  305 non-null    object
 1   link      305 non-null    object
dtypes: object(2)
memory usage: 7.1+ KB


## Open chrome and GG map

In [6]:
# create a new Chrome browser instance
service = Service(executable_path='D:\Downloads\chromedriver_win32')
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service = service, options=options)
# navigate to the application home page
url = 'https://www.google.com/maps'
driver.get(url)
time.sleep(5)

## Crawl

In [7]:
def getResInfo(page_source, id):
    soup = BeautifulSoup(page_source, 'html.parser')
    # get restaurant info
    name = soup.find('h1').text
    try:
        rating = soup.find('div', class_ = 'F7nice').find('span').find('span').text
    except:
        rating = np.nan
    try:
        rating_count = soup.find('div', class_ = 'F7nice').find_all('span')[10].text
        rating_count = re.findall(r'\d+', rating_count)[0]
    except:
        rating_count = 0
    address = soup.find('div', class_ = 'rogA2c').find('div').text
    item = pd.DataFrame({'id': id ,'name': name, 'rating': rating, 'rating_count': rating_count, 'address': address}, index = [0])
    return item




In [8]:
def getReview(res_id):
    df_review = pd.DataFrame(columns=['res_id', 'review', 'rating'])
    # click view more to view full of review
    more_buttons = driver.find_elements(By.CLASS_NAME, 'w8nwRe')
    for button in more_buttons:
        button.click()
        time.sleep(1)
    # get review
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    reviews = soup.find_all('div', class_ = 'jJc9Ad')
    for div in reviews:
        rating = div.find('div',class_ = 'DU9Pgb').find('span').get('aria-label').split(' ')[0]
        try:
            review = div.find('div', class_='MyEned').find('span').text
            
        except:
            review = '' #Someone just rate without review
        item = pd.DataFrame({'res_id': res_id, 'review': review, 'rating': rating}, index = [0])
        df_review = pd.concat([df_review, item], axis = 0)
    return df_review



In [169]:
for link in df['link'][:2]:
    print(link)

https://www.google.com/maps/place/The+Little+Door/data=!4m7!3m6!1s0x31752facfe18575b:0xd4877dbf5e55b6ad!8m2!3d10.787121!4d106.692198!16s%2Fg%2F11gmwsb3p6!19sChIJW1cY_qwvdTERrbZVXr99h9Q?authuser=0&hl=en&rclk=1
https://www.google.com/maps/place/C%C3%B4n+S%C6%A1n+Restaurant+%26+Lounge/data=!4m7!3m6!1s0x31752fac4bc9f27d:0x7f78b7bf9cd3d81f!8m2!3d10.7843201!4d106.6850108!16s%2Fg%2F11rp0hrv8z!19sChIJffLJS6wvdTERH9jTnL-3eH8?authuser=0&hl=en&rclk=1


In [12]:
res_id = 139
for link in tqdm(df['link'][res_id:]): 
    res_id += 1
    df_restaurant = pd.DataFrame(columns=['id','name', 'rating', 'rating_count', 'address'])
    df_review = pd.DataFrame(columns=['res_id', 'review', 'rating'])
    driver.get(link)
    time.sleep(5)
    #get restaurant info
    page_source = driver.page_source
    df_restaurant = pd.concat([df_restaurant, getResInfo(page_source, res_id)], axis = 0)
    #scroll down
    SCROLL_PAUSE_TIME = 5
    # Get and choose review tab and skip restaurant not having review
    if(len(driver.find_elements(By.CLASS_NAME, 'hh2c6'))==3): 
        tab = driver.find_elements(By.CLASS_NAME, 'hh2c6')[1]
    else:
        continue
    # if(driver.find_elements(By.CLASS_NAME, 'hh2c6')):
    #     tab = driver.find_elements(By.CLASS_NAME, 'hh2c6')[1]
    # else:
    #     continue #skip some restaurant doesn't have review
    tab.click()
    time.sleep(SCROLL_PAUSE_TIME)
    if(page_source == driver.page_source):
        time.sleep(SCROLL_PAUSE_TIME)
    else:
        # scroll down
        button = driver.find_element(By.CLASS_NAME, 'g88MCb')
        for i in range(0, 15):
            for j in range(0, 10):
                button.send_keys(Keys.PAGE_DOWN) #press PAGE_DOWN
            time.sleep(SCROLL_PAUSE_TIME) #wait for loading
        # scrape reviews
        page_source = driver.page_source
        df_review = pd.concat([df_review, getReview(res_id)], axis = 0)
    if(res_id == 1):
        header = True
    else:
        header = False
    df_restaurant.to_csv('data/restaurants.csv', index = False, header=header, mode='a')
    df_review.to_csv('reviews.csv', index = False, header=header, mode='a')
    


  0%|          | 0/166 [00:00<?, ?it/s]

 18%|█▊        | 30/166 [1:26:17<6:31:11, 172.58s/it]


KeyboardInterrupt: 

In [176]:
df_review.head()

,res_id,review,rating
0,1,"A very large place with spacious yard outdoor,...",3
0,1,"Based on 80_84 Tran Quoc Thao Str, District 3 ...",4
0,1,Great place to hang out with friends at weeken...,5
0,1,"The place looks fancy from outside, so the pri...",4
0,1,"Great food, fine dining. Beautiful private din...",5


In [114]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
    # click view more to view full of review
more_buttons = driver.find_elements(By.CLASS_NAME, 'w8nwRe')
for button in more_buttons:
    button.click()
    time.sleep(1)
# get review
reviews = soup.find_all('div', class_ = 'jJc9Ad')
review = reviews[3]
review.find('div', class_='MyEned').find('span').text

'The place looks fancy from outside, so the price. I was not be impressed by the food itself but mostly the hospitality of the staffs from the entrance till we finished our dinner. 4 stars!'